In [1]:
#Context: Within my former company, which was a cloud communications platform, we charge customers for each SMS text 
#sent out to their end users. Prices vary by country. My team dealt with custom pricing (discounts) for a lot for our 
#customers, especially when they push large volumes of traffic on our platform.

#Data: I will use the company's customer data, which included the price & volumes for each country they pushed traffic in.
#To respect confidentiality, I randomly altered the customer ID, price, volume, and revenue data.

#Problem Statement: To create a function that easily determines and/or predicts an appropriate price for each country, 
#given the volume.

#Who will use my result: My team, as part of day to day operations.

#Success Metrics: Make sure all countries have correct linear regression outputs.

#Uncertainty/risk #1: Not a fully accurate depiction of market rates
#Uncertainty/risk #2: Prices predicted could be below cost

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
dataset = pd.read_csv('ProjectDataset3.csv')

In [3]:
#checking null values per column
print(dataset.isnull().sum())

Country Code.    651
Customer ID      220
Volume             0
Revenue            0
Price P.U          0
dtype: int64


In [4]:
#Cleaning the data

#Show only non-NaN values in Price P.U

#Filter for non-0 values in Price P.U. Sometimes customers receive free pricing for unique reasons, 
#but this will never be applicable to an ordinary customer

#remove all Countries that are labelled [Unmapped]
clean_data = dataset.loc[(pd.notnull(dataset['Price P.U'])) & 
                         (dataset['Price P.U']!= 0) & 
                         (dataset['Country Code.']!='[Unmapped]')]

In [5]:
clean_data

,Country Code.,Customer ID,Volume,Revenue,Price P.U
1008,AD,0ab72f2,365,15.40300,0.04220
1009,AD,b8aac85,1,0.01650,0.01650
1010,AD,0bc55c3,4,0.07200,0.01800
1011,AD,bdea245,2,0.17240,0.08620
1012,AD,0c2cec5,7,0.12600,0.01800
...,...,...,...,...,...
330167,ZW,d520dad,163,4.57867,0.02809
330168,ZW,fdd945f,49,1.44844,0.02956
330169,ZW,ef14bd6,1,0.02620,0.02620
330170,ZW,ff0616c,1,0.02770,0.02770


In [6]:
#starting to get into linear regression from this moment onwards
Volume = np.array(clean_data['Volume']).reshape(-1,1)
Price = np.array(clean_data['Price P.U']).reshape(-1,1)

In [7]:
#testing out linear regression on the entire dataset
lr = LinearRegression()
lr.fit(Volume,Price)
predictions = lr.predict(Volume)

m = lr.coef_[0][0]
b = lr.intercept_[0]
print(f'y ={m}x + {b}')

#The price generally decreases by -1.34e-10x whenever volume increases by 1, which follows our intuition that prices
#should decrease as volume increases. Let's proceed with creating a separate equation for each country

y =-1.3424736546558944e-10x + 0.04517501805857787


In [8]:
#Iterate the creation of y = mx+b equations for each country 
country_codes = set(clean_data['Country Code.'].values)
country_lrs = {}
for cc in country_codes:
    country_data = clean_data[clean_data['Country Code.'] == cc]
    country_volume = country_data['Volume'].values.reshape(-1,1)
    country_price = country_data['Price P.U'].values.reshape(-1,1)
    if len(country_volume) > 0:
        country_lr = LinearRegression()
        country_lr.fit(country_volume, country_price)
        country_lrs[cc] = country_lr

In [9]:
#Observe the result from the loop
for i in country_lrs:
    print(f'{i}: y = {country_lrs[i].coef_}x + {country_lrs[i].intercept_}')

NI: y = [[-9.70940911e-08]]x + [0.05111331]
TH: y = [[-3.77856654e-10]]x + [0.02179446]
TT: y = [[-1.04368603e-08]]x + [0.0308123]
UZ: y = [[-8.83648475e-08]]x + [0.08682901]
AS: y = [[-5.13103772e-07]]x + [0.02279126]
PT: y = [[-3.99794355e-09]]x + [0.04193354]
ET: y = [[-8.75371399e-08]]x + [0.06001225]
HK: y = [[-1.62347164e-09]]x + [0.04163816]
AF: y = [[-4.62230745e-08]]x + [0.07400025]
JP: y = [[-2.07126279e-09]]x + [0.06546802]
AG: y = [[-1.69538538e-06]]x + [0.03227349]
TL: y = [[-4.39885152e-07]]x + [0.07502348]
CO: y = [[-8.25853277e-10]]x + [0.03976045]
BW: y = [[-7.38920607e-07]]x + [0.07617864]
CL: y = [[-8.50791484e-09]]x + [0.04675044]
PA: y = [[-4.650171e-08]]x + [0.05103923]
EG: y = [[-7.2163788e-10]]x + [0.06140312]
GH: y = [[-4.72112214e-09]]x + [0.040196]
IL: y = [[-4.17799245e-09]]x + [0.01911432]
BF: y = [[-1.64476066e-07]]x + [0.03346747]
UY: y = [[-9.65436048e-08]]x + [0.07413918]
PM: y = [[-2.47995916e-06]]x + [0.09752337]
KN: y = [[-1.15062303e-06]]x + [0.0445

In [10]:
#create a function that outputs the predicted price based on country and volume
#additionally, i want to make a floor for each country, otherwise the regression equation can go below 0 if the volumes
#are high enough. let's spend some time setting the floor to the min price per country
def predictPrice(country, volume):
    if country not in country_lrs:
        return -1
    
    lr = country_lrs[country]
    m = lr.coef_[0][0]
    b = lr.intercept_[0]
    if (m * volume + b) < clean_data[clean_data['Country Code.'] == country]['Price P.U'].min():
        suggprice = clean_data[clean_data['Country Code.'] == country]['Price P.U'].min()
    else:
        suggprice = m * volume + b
    
    return print(f'{country} Suggested Price (EUR): {round(suggprice,5)}')

In [11]:
#test out some country and volume combinations
predictPrice('US', 100000000)
predictPrice('TH', 10000)

#test out the floor once volumes reach a certain threshold
print('\nMin Price for GB: ', clean_data[clean_data['Country Code.'] == 'GB']['Price P.U'].min())
predictPrice('GB',10000000000000)

US Suggested Price (EUR): 0.00412
TH Suggested Price (EUR): 0.02179

Min Price for GB:  0.0006
GB Suggested Price (EUR): 0.0006


In [12]:
#That concludes my project. If I had more time/knowledge, improvements / areas to explore would be:
#1: Make the regression model update real-time, using new data that flows in.
#2: Add some form of Margin metric to the suggested price, or else we don't know how much money we're making. 
#Unfortunately, I do not have access to cost data.
#3: Each country has a separate set of market rates. In my former company, there were country/region experts that were
#able to determine the most competitive rates for each country. If I had more time, I would've collaborated with these
#experts to determine the 'real' floor, instead of relying on an inaccurate 'min' price.